In [14]:
from sqlalchemy.orm import Session
from core.config import settings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from core.prompts import STORY_PROMPT
from models.story import Story, StoryNode
from core.models import StoryLLMResponse, StoryNodeLLM


In [15]:
# Tạo LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    api_key=settings.GEMINI_API_KEY
)

# Pydantic parser cho schema StoryLLMResponse
story_parser = PydanticOutputParser(pydantic_object=StoryLLMResponse)


In [16]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        STORY_PROMPT
    ),
    (
        "human",
        "Create the story with this theme: {theme}"
    )
]).partial(format_instructions=story_parser.get_format_instructions())

In [17]:
prompt.format_prompt(theme="A hero's journey to save a village from a dragon")

ChatPromptValue(messages=[SystemMessage(content='\n                You are a creative story writer that creates engaging choose-your-own-adventure stories.\n                Generate a complete branching story with multiple paths and endings in the JSON format I\'ll specify.\n\n                The story should have:\n                1. A compelling title\n                2. A starting situation (root node) with 2-3 options\n                3. Each option should lead to another node with its own options\n                4. Some paths should lead to endings (both winning and losing)\n                5. At least one path should lead to a winning ending\n\n                Story structure requirements:\n                - Each node should have 2-3 options except for ending nodes\n                - The story should be 3-4 levels deep (including root node)\n                - Add variety in the path lengths (some end earlier, some later)\n                - Make sure there\'s at least one winning

In [51]:
raw_response = llm.invoke(prompt.invoke({"theme": "Hero aventures"}))
raw_response.content

'```json\n{"title": "The Sunken City\'s Secret", "rootNode": {"content": "You are a renowned adventurer, known for your bravery and cunning. You\'ve received a cryptic map leading to the Sunken City of Eldoria, rumored to hold a powerful artifact. You stand at the shore, ready to begin your quest. The salty air stings your face, and the waves crash gently against the sand. What do you do?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Follow the map and dive into the ocean to find the city.", "nextNode": {"content": "You descend into the depths, the sunlight fading as you go. The map leads you to a hidden entrance. Inside, you find a chamber guarded by ancient automatons. They begin to attack. Do you...", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Fight the automatons.", "nextNode": {"content": "You engage in a fierce battle, your sword clashing against their metal bodies. After a grueling fight, you manage to disable them. You proceed deep

In [52]:
raw_response

AIMessage(content='```json\n{"title": "The Sunken City\'s Secret", "rootNode": {"content": "You are a renowned adventurer, known for your bravery and cunning. You\'ve received a cryptic map leading to the Sunken City of Eldoria, rumored to hold a powerful artifact. You stand at the shore, ready to begin your quest. The salty air stings your face, and the waves crash gently against the sand. What do you do?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Follow the map and dive into the ocean to find the city.", "nextNode": {"content": "You descend into the depths, the sunlight fading as you go. The map leads you to a hidden entrance. Inside, you find a chamber guarded by ancient automatons. They begin to attack. Do you...", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Fight the automatons.", "nextNode": {"content": "You engage in a fierce battle, your sword clashing against their metal bodies. After a grueling fight, you manage to disable them

In [57]:

def post_process_response(raw_output: str) -> StoryLLMResponse:
  import json
  import re

  if raw_output.startswith("```json"):
      raw_output = raw_output[len("```json"):]

  raw_output = raw_output.strip()  # bỏ \n đầu/cuối
  if raw_output.endswith("```"):
      raw_output = raw_output[:-3].strip()

  # Clean the JSON string
  cleaned_json = raw_output.replace("\\'", "'")  # Remove escape characters before single quotes
  cleaned_json = re.sub(r'[^\x20-\x7E]', '', cleaned_json)  # Remove non-printable characters

  #Parse
  data = json.loads(cleaned_json)
  pretty_json = json.dumps(data, indent=4)
  return pretty_json

In [64]:
raw_response

AIMessage(content='```json\n{"title": "The Sunken City\'s Secret", "rootNode": {"content": "You are a renowned adventurer, known for your bravery and cunning. You\'ve received a cryptic map leading to the Sunken City of Eldoria, rumored to hold a powerful artifact. You stand at the shore, ready to begin your quest. The salty air stings your face, and the waves crash gently against the sand. What do you do?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Follow the map and dive into the ocean to find the city.", "nextNode": {"content": "You descend into the depths, the sunlight fading as you go. The map leads you to a hidden entrance. Inside, you find a chamber guarded by ancient automatons. They begin to attack. Do you...", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Fight the automatons.", "nextNode": {"content": "You engage in a fierce battle, your sword clashing against their metal bodies. After a grueling fight, you manage to disable them

In [65]:
type(raw_response)

langchain_core.messages.ai.AIMessage

In [ ]:
ans = post_process_response(raw_response.content)
ans

In [72]:
# 1. Loại bỏ code block
raw_output = raw_response.content

if raw_output.startswith("```json"):
    raw_output = raw_output[len("```json"):]

raw_output = raw_output.strip()  # bỏ \n đầu/cuối
if raw_output.endswith("```"):
    raw_output = raw_output[:-3].strip()

raw_output[3135:3140]

']}]}}'

In [ ]:



import json
import re

# The JSON string
#json_str = '{"title": "The Sunken City\'s Secret", "rootNode": {"content": "You are a seasoned adventurer, renowned for your courage and cunning. You\'ve received a cryptic map hinting at the location of the Sunken City of Eldoria, a place rumored to hold unimaginable treasures and forgotten magic. You stand at the edge of the Whispering Sea, ready to begin your quest. What do you do?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Set sail immediately, trusting your instincts and the map.", "nextNode": {"content": "The journey across the Whispering Sea is fraught with peril. Violent storms, monstrous sea creatures, and treacherous currents test your resolve. After weeks at sea, you finally reach the coordinates marked on the map. You find a submerged city. What do you do?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Dive into the city, exploring the ruins in search of treasure.", "nextNode": {"content": "You descend into the depths, navigating the eerie, waterlogged streets of Eldoria. You discover a hidden chamber guarded by a colossal, ancient kraken. The kraken attacks! Do you fight or flee?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Fight the kraken, using your skills and weapons.", "nextNode": {"content": "You engage in a fierce battle with the kraken. After a grueling struggle, you manage to defeat the beast. Exhausted but victorious, you enter the chamber. Inside, you find the Heart of Eldoria, a powerful artifact that grants you immense power. You have won!", "isEnding": true, "isWinningEnding": true}}, {"text": "Flee the kraken, attempting to escape the city.", "nextNode": {"content": "You attempt to flee, but the kraken is too fast. It crushes your ship with its tentacles. You drown in the depths.", "isEnding": true, "isWinningEnding": false}}]}}, {"text": "Attempt to bypass the city, searching for another entrance.", "nextNode": {"content": "You search for another entrance to the city but are ambushed by a rival group of adventurers, also seeking the treasure. They are well-equipped and outnumber you. You\'re captured.", "isEnding": true, "isWinningEnding": false}}]}}, {"text": "Gather more information, consulting with local scholars and sailors before setting out.", "nextNode": {"content": "You spend weeks gathering information, learning about the dangers of the sea and the rumored guardians of Eldoria. You discover that a specific type of amulet can ward off the kraken. Armed with this knowledge, you set sail. You arrive at the Sunken City, and find the location of the amulet. Do you enter the city?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Enter the city, with the amulet.", "nextNode": {"content": "You descend into the depths, the amulet protecting you from the kraken\'s attacks. You navigate the ruins, finding the Heart of Eldoria and claiming it without a fight. You have won!", "isEnding": true, "isWinningEnding": true}}, {"text": "Attempt to find the main entrance without the amulet.", "nextNode": {"content": "You attempt to enter the city without the amulet, and are immediately attacked by the kraken. You are defeated.", "isEnding": true, "isWinningEnding": false}}]}}, {"text": "Hire a skilled crew and a sturdy ship, preparing for a long voyage.", "nextNode": {"content": "You assemble a skilled crew and a durable ship, preparing for the long journey. During the voyage, your ship is attacked by pirates. Do you fight or attempt to negotiate?", "isEnding": false, "isWinningEnding": false, "options": [{"text": "Fight the pirates.", "nextNode": {"content": "A fierce battle ensues. Your crew is skilled, but the pirates are ruthless. After a long fight, the pirates overwhelm you and your crew. You are captured, and your ship is plundered.", "isEnding": true, "isWinningEnding": false}}, {"text": "Attempt to negotiate with the pirates.", "nextNode": {"content": "You attempt to negotiate, offering a share of your treasure in exchange for safe passage. The pirates agree, but betray you once you reach the Sunken City. They leave you stranded, and take the treasure for themselves.", "isEnding": true, "isWinningEnding": false}}]}}]}}'
json_str = raw_output
# Clean the JSON string
cleaned_json = json_str.replace("\\'", "'")  # Remove escape characters before single quotes
cleaned_json = re.sub(r'[^\x20-\x7E]', '', cleaned_json)  # Remove non-printable characters

data = json.loads(cleaned_json)
pretty_json = json.dumps(data, indent=4)
print(pretty_json)

JSONDecodeError: Expecting ',' delimiter: line 1 column 3138 (char 3137)

In [23]:
raw_response = pretty_json
response_text = raw_response

In [24]:
if hasattr(raw_response, "content"):
    response_text = raw_response.content

story_structure = story_parser.parse(response_text)

In [26]:
story_structure.rootNode

StoryNodeLLM(content="You are a seasoned adventurer, renowned for your courage and cunning. You've received a cryptic map hinting at the location of the Sunken City of Eldoria, a place rumored to hold unimaginable treasures and forgotten magic. You stand at the edge of the Whispering Sea, ready to begin your quest. What do you do?", isEnding=False, isWinningEnding=False, options=[StoryOptionLLM(text='Set sail immediately, trusting your instincts and the map.', nextNode={'content': 'The journey across the Whispering Sea is fraught with peril. Violent storms, monstrous sea creatures, and treacherous currents test your resolve. After weeks at sea, you finally reach the coordinates marked on the map. You find a submerged city. What do you do?', 'isEnding': False, 'isWinningEnding': False, 'options': [{'text': 'Dive into the city, exploring the ruins in search of treasure.', 'nextNode': {'content': 'You descend into the depths, navigating the eerie, waterlogged streets of Eldoria. You disco

In [39]:
from db.database import get_db, SessionLocal
db = SessionLocal()

from db.database import create_tables

create_tables()

In [40]:
story_db = Story(title=story_structure.title, session_id="abcd")
db.add(story_db)
db.flush()

In [41]:
root_node_data = story_structure.rootNode
if isinstance(root_node_data, dict):
    root_node_data = StoryNodeLLM.model_validate(root_node_data)

In [42]:
root_node_data

StoryNodeLLM(content="You are a seasoned adventurer, renowned for your courage and cunning. You've received a cryptic map hinting at the location of the Sunken City of Eldoria, a place rumored to hold unimaginable treasures and forgotten magic. You stand at the edge of the Whispering Sea, ready to begin your quest. What do you do?", isEnding=False, isWinningEnding=False, options=[StoryOptionLLM(text='Set sail immediately, trusting your instincts and the map.', nextNode={'content': 'The journey across the Whispering Sea is fraught with peril. Violent storms, monstrous sea creatures, and treacherous currents test your resolve. After weeks at sea, you finally reach the coordinates marked on the map. You find a submerged city. What do you do?', 'isEnding': False, 'isWinningEnding': False, 'options': [{'text': 'Dive into the city, exploring the ruins in search of treasure.', 'nextNode': {'content': 'You descend into the depths, navigating the eerie, waterlogged streets of Eldoria. You disco

In [43]:
root_node_data.isWinningEnding

False

In [44]:
print(StoryNode.__table__.columns.keys())


['id', 'story_id', 'content', 'is_root', 'is_ending', 'is_winning_ending', 'options']


In [45]:
def process_story_node(db: Session, story_id: int, node_data: StoryNodeLLM, is_root: bool = False) -> StoryNode:
      node = StoryNode(
          story_id=story_id,
          content=node_data.content if hasattr(node_data, "content") else node_data["content"],
          is_root=is_root,
          is_ending=node_data.isEnding if hasattr(node_data, "isEnding") else node_data["isEnding"],
          is_winning_ending=node_data.isWinningEnding if hasattr(node_data, "isWinningEnding") else node_data["isWinningEnding"],
          options=[]
      )
      db.add(node)
      db.flush()

      if not node.is_ending and (hasattr(node_data, "options") and node_data.options):
          options_list = []
          for option_data in node_data.options:
              next_node = option_data.nextNode

              if isinstance(next_node, dict):
                  next_node = StoryNodeLLM.model_validate(next_node)

              child_node = process_story_node(db, story_id, next_node, False)

              options_list.append({
                  "text": option_data.text,
                  "node_id": child_node.id
              })

          node.options = options_list

      db.flush()
      return node

In [46]:
process_story_node(db, story_db.id, root_node_data, is_root=True)
db.commit()
story_db

In [ ]:
class StoryGenerator:

  @classmethod
  def _get_llm(cls):
    return ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", api_key=settings.GEMINI_API_KEY)
  
  @classmethod
  def generate_story(cls, db: Session, session_id: str, theme: str = "fantasy")-> Story:
      llm = cls._get_llm()
      story_parser = PydanticOutputParser(pydantic_object=StoryLLMResponse)

      prompt = ChatPromptTemplate.from_messages([
          (
              "system",
              STORY_PROMPT
          ),
          (
              "human",
              f"Create the story with this theme: {theme}"
          )
      ]).partial(format_instructions=story_parser.get_format_instructions())

      raw_response = llm.invoke(prompt.invoke({}))

      response_text = raw_response
      if hasattr(raw_response, "content"):
          response_text = raw_response.content

      story_structure = story_parser.parse(response_text)

      story_db = Story(title=story_structure.title, session_id=session_id)
      db.add(story_db)
      db.flush()

      root_node_data = story_structure.rootNode
      if isinstance(root_node_data, dict):
          root_node_data = StoryNodeLLM.model_validate(root_node_data)

      cls._process_story_node(db, story_db.id, root_node_data, is_root=True)

      db.commit()
      return story_db

  @classmethod
  def _process_story_node(cls, db: Session, story_id: int, node_data: StoryNodeLLM, is_root: bool = False) -> StoryNode:
      node = StoryNode(
          story_id=story_id,
          content=node_data.content if hasattr(node_data, "content") else node_data["content"],
          is_root=is_root,
          is_ending=node_data.isEnding if hasattr(node_data, "isEnding") else node_data["isEnding"],
          is_winning_ending=node_data.isWinningEnding if hasattr(node_data, "isWinningEnding") else node_data["isWinningEnding"],
          options=[]
      )
      db.add(node)
      db.flush()

      if not node.is_ending and (hasattr(node_data, "options") and node_data.options):
          options_list = []
          for option_data in node_data.options:
              next_node = option_data.nextNode

              if isinstance(next_node, dict):
                  next_node = StoryNodeLLM.model_validate(next_node)

              child_node = cls._process_story_node(db, story_id, next_node, False)

              options_list.append({
                  "text": option_data.text,
                  "node_id": child_node.id
              })

          node.options = options_list

      db.flush()
      return node
